In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9-0
!apt install gcc-6
!apt install g++-6
!ln -s /usr/bin/gcc-6 /usr/local/cuda/bin/gcc
!ln -s /usr/bin/g++-6 /usr/local/cuda/bin/g++



Reading package lists... Done
Building dependency tree       
Reading state information... Done
Correcting dependencies... Done
The following additional packages will be installed:
  nvidia-396
The following NEW packages will be installed:
  nvidia-396
0 upgraded, 1 newly installed, 0 to remove and 4 not upgraded.
228 not fully installed or removed.
Need to get 80.0 MB of archives.
After this operation, 328 MB of additional disk space will be used.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  nvidia-396 396.44-0ubuntu1 [80.0 MB]
Fetched 80.0 MB in 2s (35.6 MB/s)
(Reading database ... 78035 files and directories currently installed.)
Preparing to unpack .../nvidia-396_396.44-0ubuntu1_amd64.deb ...
Unpacking nvidia-396 (396.44-0ubuntu1) ...
dpkg: error processing archive /var/cache/apt/archives/nvidia-396_396.44-0ubuntu1_amd64.deb (--unpack):
 trying to overwrite '/usr/share/egl/egl_external_platform.d/10_nvidia_wayland.json', which is also in package

In [ ]:
#!nvidia-smi
!apt-get install cuda

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  acpid adwaita-icon-theme at-spi2-core ca-certificates-java cuda-9-2
  cuda-command-line-tools-9-2 cuda-compiler-9-2 cuda-cublas-dev-9-2
  cuda-cudart-9-2 cuda-cudart-dev-9-2 cuda-cufft-dev-9-2 cuda-cuobjdump-9-2
  cuda-cupti-9-2 cuda-curand-dev-9-2 cuda-cusolver-dev-9-2
  cuda-cusparse-dev-9-2 cuda-demo-suite-9-2 cuda-documentation-9-2
  cuda-driver-dev-9-2 cuda-drivers cuda-gdb-9-2 cuda-gpu-library-advisor-9-2
  cuda-libraries-dev-9-2 cuda-memcheck-9-2 cuda-misc-headers-9-2
  cuda-npp-dev-9-2 cuda-nsight-9-2 cuda-nvcc-9-2 cuda-nvdisasm-9-2
  cuda-nvgraph-dev-9-2 cuda-nvml-dev-9-2 cuda-nvprof-9-2 cuda-nvprune-9-2
  cuda-nvrtc-dev-9-2 cuda-nvvp-9-2 cuda-runtime-9-2 cuda-samples-9-2
  cuda-toolkit-9-2 cuda-tools-9-2 cuda-visual-tools-9-2 dbus
  dconf-gsettings-backend dconf-service default-jre default-jre-headless dkms
  dmsetup fakeroot f

In [ ]:
!/usr/local/cuda/bin/nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Tue_Jun_12_23:07:04_CDT_2018
Cuda compilation tools, release 9.2, V9.2.148


In [ ]:
%%writefile trabalin.cu

/* TÃ³picos Especiais em ProgramaÃ§Ã£o Paralela (ProgramaÃ§Ã£o Paralela com GPU) */
/* 2018/2 */
/* Codigo: Soma de vetores em CUDA */

#include <cstdio>
#include <algorithm>

using namespace std;

//para tomada de tempo
#include <sys/time.h>
//retorna o instante de tempo atual em segundos
double tempoAtual() {
   struct timespec tempo;
   clock_gettime(CLOCK_MONOTONIC_RAW, &tempo);
   return tempo.tv_sec + tempo.tv_nsec/1000000000.0;               
}

//para checar erros chamadas Cuda
#define CUDA_SAFE_CALL(call) { \
   cudaError_t err = call;     \
   if(err != cudaSuccess) {    \
      fprintf(stderr,"Erro no arquivo '%s', linha %i: %s.\n",__FILE__, __LINE__,cudaGetErrorString(err)); \
      exit(EXIT_FAILURE); } } 

//funcao para execucao sequencial na CPU (os tres vetores possuem o mesmo tamanho n)
void calculaDist(const double x[], const double y[], const double z[], double c[], int n, double q, double w, double e) {
   int i;
   for(i=0; i<n; i++) 
      c[i] = sqrt( (x[i] - q)*(x[i] - q) + (y[i] - w)*(y[i] - w) + (z[i] - e) * (z[i] - e) );
}

//funcao para calcular o maximo
void calculaMax(double h[], int n) {
    double _max = -1.0;
    for (int i = 0; i < n; i++) {
        if (h[i] > _max) {
            _max = h[i];
        }
    }
    
    printf("A maior distancia eh: %f", _max);
}

//kernel para execucao paralela na GPU (os tres vetores possuem o mesmo tamanho n)
__global__ void calculaDistPar(const double *x, const double *y, const double *z, double *c, int n, double q, double w, double e) {
   int i = blockIdx.x * blockDim.x + threadIdx.x;
   if(i < n) 
      c[i] = sqrt( (x[i] - q)*(x[i] - q) + (y[i] - w)*(y[i] - w) + (z[i] - e) * (z[i] - e) );
   
}


//kernel para execucao paralela em CUDA (!!assume-se que o tamanho do vetor sera sempre potencia de 2!!)
__global__ void calculaMaxPar(double *a, double *b) {
   int i = blockIdx.x * blockDim.x + threadIdx.x;
   int n = blockDim.x;
   int offset;

   for(offset=1; offset<n; offset*=2) {
      double aux;
      if(threadIdx.x >= offset) {
         aux = a[i-offset]; 
      }
      __syncthreads();
      if(threadIdx.x >= offset) {          
         a[i] = max(aux, a[i]);
         
          //if (threadIdx.x == n-1) printf("%f ", a[i]);
      }
      __syncthreads();
   }
   if (threadIdx.x == (n-1)) //ultima thread do bloco, salva seu valor no outo vetor
      b[blockIdx.x] = a[i];
}



//funcao principal
int main(int argc, char** argv) {
   double *h_x, *d_x;
   double *h_y, *d_y;
   double *h_z, *d_z;
   double *h_c, *d_c;
   double *h_b,*d_b;
   int n, n_bytes;
   double q,w,e;
   //para medidas de tempo
   double inicio, fim, inicio2,fim2;
   double tempo_seq,tempo_seq2, tempo_ida, tempo_volta;
   cudaEvent_t start, stop;

   //verifica os parametros de entrada
   if(argc < 5) {
      printf("Digite: %s <tamanho dos vetores> <x> <y> <z>\n", argv[0]);
      exit(EXIT_FAILURE);
   }
   //armazena os parametros de entrada
   n = atol(argv[1]);
   q = atol(argv[2]);
   w = atol(argv[3]);
   e = atol(argv[4]);
   //calcula o tamanho em bytes dos vetores
   n_bytes = n * sizeof(double);

   //aloca memoria na CPU (host)
   h_x = (double*) malloc (n_bytes);
   if (h_x == NULL) { 
      printf("Erro malloc vetor X\n"); 
      exit(EXIT_FAILURE);
   }
   h_y = (double*) malloc (n_bytes);
   if (h_y == NULL) { 
      printf("Erro malloc vetor Y\n"); 
      exit(EXIT_FAILURE);
   }
   h_z = (double*) malloc (n_bytes);
   if (h_z == NULL) { 
      printf("Erro malloc vetor Z\n"); 
      exit(EXIT_FAILURE);
   }
    
   h_c = (double*) malloc (n_bytes);
   if (h_c == NULL) { 
      printf("Erro malloc vetor C\n"); 
      exit(EXIT_FAILURE);
   }
    
   h_b = (double*) malloc (n_bytes);
   if (h_c == NULL) { 
      printf("Erro malloc vetor B\n"); 
      exit(EXIT_FAILURE);
   }

   //inicializa os dois vetores de entrada com o valor 1.0
   for(int i=0; i<n; i++) {
      h_x[i] =  2.0 + i;
      h_y[i] =  2.0 + i;
      h_z[i] = 2.0 + i;
   }

   //!!! ------------------------ executa sequencial ---------------------------------- !!!//
   inicio = tempoAtual();
   calculaDist(h_x, h_y, h_z, h_c, n, q, w, e);
   fim = tempoAtual();
   tempo_seq = fim-inicio; // em segundos

   //!!! ------------------------ executa em paralelo em CUDA -------------------------- !!!//
   //dispara o kernel 
   inicio = tempoAtual();
   int n_threads = 1024; //numero de threads por bloco
   int n_blocos = (n + n_threads-1) / n_threads; //numero de blocos
   
    
   //aloca espaco para os vetores na GPU
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_x, n_bytes));
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_y, n_bytes));
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_z, n_bytes));
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_c, n_bytes));
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_b, n_bytes));
   
    

   //copia os vetores de entrada da CPU para a GPU (host para device)
   CUDA_SAFE_CALL(cudaMemcpy(d_x, h_x, n_bytes, cudaMemcpyHostToDevice));
   CUDA_SAFE_CALL(cudaMemcpy(d_y, h_y, n_bytes, cudaMemcpyHostToDevice));
   CUDA_SAFE_CALL(cudaMemcpy(d_z, h_z, n_bytes, cudaMemcpyHostToDevice));

   printf("Kernel com %d bloco(s) de %d threads:\n", n_blocos, n_threads);
   fim = tempoAtual();
   tempo_ida = fim-inicio; // em segundos

   //mede o tempo de execucao do kernel apenas
   CUDA_SAFE_CALL(cudaEventCreate(&start));
   CUDA_SAFE_CALL(cudaEventCreate(&stop));
   //inicia a contagem de tempo
   CUDA_SAFE_CALL(cudaEventRecord(start));
   //dispara o kernel 
   calculaDistPar <<<n_blocos, n_threads>>> (d_x, d_y, d_z, d_c, n, q, w, e);
   calculaMaxPar <<<n_blocos, n_threads>>> (d_c,d_b);
   CUDA_SAFE_CALL(cudaGetLastError());
   CUDA_SAFE_CALL(cudaEventRecord(stop));
   CUDA_SAFE_CALL(cudaEventSynchronize(stop));
   //finaliza a contagem de tempo 
   float delta_eventos = 0;
   //(tempo  decorrido entre o registro do event start e do evento stop, em milisegundos)
   CUDA_SAFE_CALL(cudaEventElapsedTime(&delta_eventos, start, stop));

   inicio=tempoAtual();
   //copia resultado da GPU para a CPU (device para host)
   CUDA_SAFE_CALL(cudaMemcpy(h_c, d_c, n_bytes, cudaMemcpyDeviceToHost)) 
   CUDA_SAFE_CALL(cudaMemcpy(h_b, d_b, n_bytes, cudaMemcpyDeviceToHost)) 
   fim = tempoAtual();
   tempo_volta = fim-inicio; // em segundos
  
   //verifica se o resultado esta correto
   for (int i = 0; i < n; ++i) {
       if (fabs(sqrt((h_x[i] - q)*(h_x[i] - q) + (h_y[i] - w)*(h_y[i] - w) + (h_z[i] - e) * (h_z[i] - e)) - h_c[i]) > 1e-5) {
            fprintf(stderr, "resultado incorreto para o elemento de indice %d!\n", i);
            exit(EXIT_FAILURE);
        }
   }
    
   //!!! ------------------------ executa sequencial 2 --------------------------------- !!!//
   inicio2 = tempoAtual();
   calculaMax(h_c, n);
   fim2 = tempoAtual();
   tempo_seq2 = fim2-inicio2; // em segundos
   tempo_seq = tempo_seq + tempo_seq2  ;
    
     
 
   printf("\nValor max gpu: %f\n", *max_element(h_b,h_b+n_blocos));

    
   printf("PASSOU NO TESTE\n");
 
   //libera a memoria na GPU
   CUDA_SAFE_CALL(cudaFree(d_x))
   CUDA_SAFE_CALL(cudaFree(d_y))
   CUDA_SAFE_CALL(cudaFree(d_z))
   CUDA_SAFE_CALL(cudaFree(d_c))
   //libera a memoria na CPU
   free(h_x);
   free(h_y);
   free(h_z);
   free(h_c);

   //------------------------------- imprime dos tempos de execucao ----------------------//
   printf("Tempo sequencial = %g seg \n", tempo_seq);
   printf("Tempo paralelo   = %f seg \n\n", delta_eventos/1000);
   printf("Speedup %f\n\n", tempo_seq / (delta_eventos/1000));
   printf("Tempo ida (kernel)   = %f seg \n", tempo_ida);
   printf("Tempo volta (kernel) = %f seg \n", tempo_volta);

   return 0;
}

Overwriting trabalin.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -o trabalin trabalin.cu

In [ ]:
!./trabalin 1048576 1.0 1.0 1.0

Kernel com 1024 bloco(s) de 1024 threads:
A maior distancia eh: 1816186.907597
Valor max gpu: 1816186.907597
PASSOU NO TESTE
Tempo sequencial = 0.0257446 seg 
Tempo paralelo   = 0.001312 seg 

Speedup 19.626231

Tempo ida (kernel)   = 0.126150 seg 
Tempo volta (kernel) = 0.011670 seg 


In [ ]:
%%writefile prefixos.cu


/* Codigo: Soma de prefixos em CUDA (um bloco de threads apenas) */

/* Para compilar: nvcc -o prefixos prefixos.cu */

#include <stdio.h>

//para checar erros chamadas Cuda
#define CUDA_SAFE_CALL(call) { \
   cudaError_t err = call;     \
   if(err != cudaSuccess) {    \
      fprintf(stderr,"Erro no arquivo '%s', linha %i: %s.\n",__FILE__, __LINE__,cudaGetErrorString(err)); \
      exit(EXIT_FAILURE); } } 

//para tomada de tempo
#include <sys/time.h>
#define BILLION 1000000000L

/* The argument now should be a double (not a pointer to a double) */
#define GET_TIME(now) { \
   struct timespec time; \
   clock_gettime(CLOCK_MONOTONIC_RAW, &time); \
   now = time.tv_sec + time.tv_nsec/1000000000.0; \
}

//funcao para execucao sequencial
void somaPrefixoSeq(float *a, int n) {
   int i;
   for(i=1; i<n; i++)
       a[i] = a[i-1] + a[i];
}

//Kernel para execucao paralela em CUDA (!!assume-se que o tamanho do vetor sera sempre potencia de 2!!)
//e que foram disparadas uma thread para cada elemento, em um unico bloco
__global__ void somaPrefixoPar(float *a) {
   int i = threadIdx.x;
   int n = blockDim.x;
   int offset;

   for(offset=1; offset<n; offset*=2) {
      int aux;
      if(i >= offset) 
         aux = a[i-offset]; 
      __syncthreads();
      if(i >= offset) 
         a[i] = aux + a[i]; 
      __syncthreads();
   }
}

//funcao para imprimir os elementos de um vetor
void printVetor(float *vetor, int n) {
   int i;
   for(i=0; i<n; i++)
      printf("%.1f ", vetor[i]);
   printf("\n");
}

//funcao principal
int main(int argc, char** argv) {
   float *h_a, *h_b, *d_a;
   int n, n_bytes;
   //para medidas de tempo
   double inicio, fim;
   double tempo_seq;
   cudaEvent_t start, stop;

   //verifica os parametros de entrada
   if(argc < 2) {
      printf("Digite: %s <tamanho do vetor (numero max de thread por bloco)> \n", argv[0]);
      exit(EXIT_FAILURE);
   }
   //armazena o parametro de entrada
   n = atol(argv[1]);
   //calcula o tamanho em bytes do vetor
   n_bytes = n * sizeof(float);

   //aloca memoria na CPU (host)
   h_a = (float*) malloc (n_bytes);
   h_b = (float*) malloc (n_bytes);
   if ((h_a == NULL) || (h_b == NULL)) {
      printf("Erro malloc vetor\n");
      exit(EXIT_FAILURE);
   }

   //inicializa o vetor de entrada com o valor 1.0
   for(int i=0; i<n; i++) {
      h_a[i] = (float) 1;
      h_b[i] = (float) 1;
   }

   //!!! ------------------------ executa sequencial ---------------------------------- !!!//
   GET_TIME(inicio);
   somaPrefixoSeq(h_b, n);
   GET_TIME(fim);
   tempo_seq = fim-inicio; // em segundos

   //!!! ------------------------ executa em paralelo em CUDA -------------------------- !!!//
   //aloca espaco para o vetor na GPU
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_a, n_bytes));

   //copia o vetor de entrada da CPU para a GPU (host para device)
   CUDA_SAFE_CALL(cudaMemcpy(d_a, h_a, n_bytes, cudaMemcpyHostToDevice));

   //dispara o kernel 
   int n_threads = n; //numero de threads por bloco
   int n_blocos = 1; //numero de blocos

   printf("kernel com %d blocos de %d threads\n", n_blocos, n_threads);
   CUDA_SAFE_CALL(cudaEventCreate(&start));
   CUDA_SAFE_CALL(cudaEventCreate(&stop));
   CUDA_SAFE_CALL(cudaEventRecord(start));
   somaPrefixoPar <<<n_blocos, n_threads>>> (d_a);
   CUDA_SAFE_CALL(cudaEventRecord(stop));
   CUDA_SAFE_CALL(cudaEventSynchronize(stop));
   CUDA_SAFE_CALL(cudaGetLastError());
   float delta_eventos = 0;
   CUDA_SAFE_CALL(cudaEventElapsedTime(&delta_eventos, start, stop));

   //copia resultado da GPU para a CPU (device para host)
   CUDA_SAFE_CALL(cudaMemcpy(h_a, d_a, n_bytes, cudaMemcpyDeviceToHost))

    //verifica se o resultado esta correto
   for (int i = 0; i < n; ++i) {
       if (fabs(h_a[i] - h_b[i]) > 1e-5) {
            fprintf(stderr, "resultado incorrento para o elemento de indice %d!\n", i);
            exit(EXIT_FAILURE);
        }
    }
    printf("PASSOU NO TESTE\n");

   //libera a memoria na GPU
   CUDA_SAFE_CALL(cudaFree(d_a))

   //libera a memoria na CPU
   free(h_a);
   free(h_b);

   //------------------------------- imprime dos tempos de execucao ----------------------//
   printf("Tempo sequencial = %g seg \n", tempo_seq);
   printf("Tempo paralelo   = %f seg \n", delta_eventos/1000);

    return 0;
}

Writing prefixos.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -o prefixos prefixos.cu

In [ ]:
!./prefixos 1024

kernel com 1 blocos de 1024 threads
PASSOU NO TESTE
Tempo sequencial = 5.12e-06 seg 
Tempo paralelo   = 0.000120 seg 


In [ ]:
%%writefile somaelementos1.cu

/* Codigo: Soma os elementos de um vetor em CUDA */

/* Para compilar: nvcc -o somaielementos1 somaelementos1.cu */

#include <stdio.h>

//para tomada de tempo
#include <sys/time.h>
#define BILLION 1000000000L
//o agumento dee ser double
#define GET_TIME(now) { \
   struct timespec time; \
   clock_gettime(CLOCK_MONOTONIC_RAW, &time); \
   now = time.tv_sec + time.tv_nsec/1000000000.0; \
  }

//para checar erros chamadas Cuda
#define CUDA_SAFE_CALL(call) { \
   cudaError_t err = call;     \
   if(err != cudaSuccess) {    \
      fprintf(stderr,"Erro no arquivo '%s', linha %i: %s.\n",__FILE__, __LINE__,cudaGetErrorString(err)); \
      exit(EXIT_FAILURE); } } 

//funcao para execucao sequencial
double somaElementosVetorSeq(float *a, long int n) {
   long int i;
   double soma=0;
   for(i=0; i<n; i++)
       soma += a[i];
   return soma;
}

//kernel para execucao paralela em CUDA (!!assume-se que o tamanho do vetor sera sempre potencia de 2!!)
__global__ void somaElementosVetorPar(float *a, float *b) {
   int i = blockIdx.x * blockDim.x + threadIdx.x;
   int n = blockDim.x;
   int offset;

   for(offset=1; offset<n; offset*=2) {
      int aux;
      if(threadIdx.x >= offset) {
         aux = a[i-offset]; 
      }
      __syncthreads();
      if(threadIdx.x >= offset) {
         a[i] = aux + a[i]; 
      }
      __syncthreads();
   }
   if (threadIdx.x == (n-1)) //ultima thread do bloco, salva seu valor no outo vetor
      b[blockIdx.x] = a[i];
}

//funcao principal
int main(int argc, char** argv) {
   float *h_s, *h_a, *d_a, *h_b, *d_b;
   double soma_seq;
   long int n, n_bytes;
   int n_threads;
   //para medidas de tempo
   double inicio, fim;
   double tempo_seq;

   //verifica os parametros de entrada
   if(argc < 3) {
      printf("Digite: %s <tamanho do vetor (deve ser potencia de 2 e maior ou igual a 256!)> <threads por bloco (deve ser potendia de 2)>\n", argv[0]);
      exit(EXIT_FAILURE);
   }
   //armazena o parametro de entrada
   n = atol(argv[1]);
   n_threads = atol(argv[2]); //numero de threads por bloco
   
   //calcula o tamanho em bytes do vetor
   n_bytes = n * sizeof(float);

   //aloca memoria na CPU (host)
   h_s = (float*) malloc (n_bytes);
   h_a = (float*) malloc (n_bytes);
   h_b = (float*) malloc (n_bytes);
   if ((h_s == NULL) || (h_a == NULL) || (h_b == NULL)) {
      printf("Erro malloc vetor\n");
      exit(EXIT_FAILURE);
   }

   //inicializa o vetor de entrada com o valor 1.0
   for(long int i=0; i<n; i++) {
      h_s[i] = (float) 1;
      h_a[i] = (float) 1;
   }

   //!!! ------------------------ executa sequencial ---------------------------------- !!!//
   GET_TIME(inicio);
   soma_seq = somaElementosVetorSeq(h_s, n);
   GET_TIME(fim);

   tempo_seq = fim-inicio; // em segundos

   //!!! ------------------------ executa em paralelo em CUDA -------------------------- !!!//
   int n_blocos1, n_blocos2; //numero de blocos

   GET_TIME(inicio);
   n_blocos1 = (n + n_threads-1) / n_threads; //numero de blocos (primeira rodada)
   
   //aloca espaco para o vetor na GPU
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_a, n_bytes));
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_b, n_blocos1 * sizeof(float)));

   double soma_par=0;
   printf("kernel com %d blocos de %d threads\n", n_blocos1, n_threads);
   //copia o vetor de entrada da CPU para a GPU (host para device)
   CUDA_SAFE_CALL(cudaMemcpy(d_a, h_a, n_bytes, cudaMemcpyHostToDevice));
   //dispara o kernel 
   somaElementosVetorPar <<<n_blocos1, n_threads>>> (d_a, d_b);
   CUDA_SAFE_CALL(cudaGetLastError());

   n_blocos2 = (n_blocos1 + n_threads-1) / n_threads; //numero de blocos (segunda rodada)
   if (n_blocos2 > 0) { //faz outra rodada na GPU       
      printf("kernel com %d blocos de %d threads\n", n_blocos2, n_threads);
      somaElementosVetorPar <<<n_blocos2, n_threads>>> (d_b, d_a);
      CUDA_SAFE_CALL(cudaGetLastError());
      //copia resultado da GPU para a CPU (device para host)
      CUDA_SAFE_CALL(cudaMemcpy(h_b, d_a, n_blocos2 * sizeof(float), cudaMemcpyDeviceToHost))
      //termina os calculos na CPU
      for (int i = 0; i < n_blocos2; i++) {
         soma_par += h_b[i];
      }
   } else {
      //copia resultado da GPU para a CPU (device para host)
      CUDA_SAFE_CALL(cudaMemcpy(h_b, d_b, n_blocos1 * sizeof(float), cudaMemcpyDeviceToHost))
      //termina os calculos na CPU
      for (int i = 0; i < n_blocos1; i++) {
         soma_par += h_b[i];
      }
   }
   GET_TIME(fim);
   
   printf("soma sequencial = %.10lf \n", soma_seq);
   printf("soma paralelo = %.10lf  \n", soma_par);
   //verifica se o resultado esta correto
   if (fabs(soma_par - soma_seq) > 1e-5) {
      fprintf(stderr, "resultado incorreto\n");
      exit(EXIT_FAILURE);
   }
   printf("PASSOU NO TESTE\n");

   //libera a memoria na GPU
   CUDA_SAFE_CALL(cudaFree(d_a));
   CUDA_SAFE_CALL(cudaFree(d_b));

   //libera a memoria na CPU
   free(h_a);
   free(h_s);
   free(h_b);

   //------------------------------- imprime dos tempos de execucao ----------------------//
   printf("Tempo sequencial = %g seg \n", tempo_seq);
   printf("Tempo paralelo = %f seg \n", (fim-inicio));

   return 0;
}

Writing somaelementos1.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -o somaelementos1 somaelementos1.cu

In [ ]:
!./somaelementos1 134217728 1024

kernel com 131072 blocos de 1024 threads
kernel com 128 blocos de 1024 threads
soma sequencial = 134217728.0000000000 
soma paralelo = 134217728.0000000000  
PASSOU NO TESTE
Tempo sequencial = 0.455809 seg 
Tempo paralelo = 0.284679 seg 


In [ ]:
%%writefile multMatrix.cu
/*
Descricao: multiplicacao de matrizes em paralelo usando GPU
Entrada: dimensao das matrizes e dos blocos de threads
Saida: tempos de execucao
Restricoes: por simplificacao, matrizes quadradas inicializadas para teste.
            A dimensao das matrizes e dos blocos deve ser potencia de 2.  
*/

#include <stdio.h>
//para tomada de tempo
#include <sys/time.h>
//o agumento dee ser double
#define GET_TIME(now) { \
   struct timespec time; \
   clock_gettime(CLOCK_MONOTONIC_RAW, &time); \
   now = time.tv_sec + time.tv_nsec/1000000000.0; \
  }

//para checar erros chamadas Cuda
#define CUDA_SAFE_CALL(call) { \
   cudaError_t err = call;     \
   if(err != cudaSuccess) {    \
      fprintf(stderr,"Erro no arquivo '%s', linha %i: %s.\n",__FILE__, __LINE__,cudaGetErrorString(err)); \
      exit(EXIT_FAILURE); } } 

//funcao para execucao sequencial
void multMatSeq(float *a, float *b, float *c, int n) {
   int i, j, k;
   float soma;
   for(i=0; i<n; i++)
      for(j=0; j<n; j++) {
         soma = 0;
         for(k=0; k<n; k++) {
            soma += a[i*n+k] * b[k*n+j];
         }
         c[i*n+j] = soma;
      }
}

//kernel para execucao paralela em CUDA (!!assume-se que o tamanho do vetor sera sempre potencia de 2!!)
//e que foram disparadas uma thread para cada elemento da matriz de saida
__global__ void multMatPar(float *a, float *b, float *c, int n) {
   //coordenadas globais da thread   
   int i = blockIdx.x * blockDim.x + threadIdx.x;
   int j = blockIdx.y * blockDim.y + threadIdx.y;

   //calcula o elemento C(i,j)
   float valor = 0;
   for (int k = 0; k < n; k++) {
      valor += a[i*n+k] * b[k*n+j];
   }
   //escreve o valor calculado na matriz de saida
   c[i*n+j] = valor;
}

//funcao que aloca espaco para uma matriz e preenche seus valores
//entrada: matriz de entrada, dimensoes da matriz
//saida: retorna 1 se a matriz foi preenchida com sucesso e 0 caso contrario
int preencheMatriz(float **mat, int linhas, int colunas) {
   int i, j;
   //aloca espaco de memoria para a matriz
   *mat = (float*) malloc(sizeof(float) * linhas * colunas);
   if (mat == NULL) return 0;
   //preenche o vetor
   for (i=0; i<linhas; i++) {
      for (j=0; j<colunas; j++) {
         *((*mat) + (i*colunas+j)) = 1.5;
      }
   }
   return 1;
}

//funcao que imprime uma matriz
//entrada: matriz de entrada, dimensoes da matriz
//saida: matriz impressa na tela
void imprimeMatriz(float *mat, int linhas, int colunas, FILE *arq) {
   int i, j;
   for (i=0; i<linhas; i++) {
      for (j=0; j<colunas; j++) {
         fprintf(arq, "%.1f ", mat[i*colunas+j]);
      }
      fprintf(arq, "\n");
   }
}

//funcao principal
int main(int argc, char** argv) {
   float *h_a, *h_b, *h_c, *h_c_par; //matrizes host
   float *d_a, *d_b, *d_c; //matrizes device
   //para medidas de tempo
   double inicio, fim;
   double tempo_seq, tempo_par_ini, tempo_par_fim;
   cudaEvent_t start, stop;
   //entrada de dados
   //FILE *arqA, *arqB, *arqC; //arquivos dos dados de entrada e saida
   unsigned int n; //dimensao da matriz de entrada
   long int n_bytes; //qtde bytes por matriz
   
   //tamanho dos blocos de threads
   unsigned int tam_bloco;

   //le e valida os parametros de entrada
   if(argc < 3) {
      fprintf(stderr, "Digite: %s <dimensao matrizes (potencia de 2)> <tamanho blocos (potencia de 2, max 32)>.\n", argv[0]);
      exit(EXIT_FAILURE);
   }
   //dimensao das matrizes e tamanho dos blocos
   n = atol(argv[1]);   
   tam_bloco = atol(argv[2]);   

   //calcula o tamanho em bytes das matrizes
   n_bytes = n*n*sizeof(float);

   //aloca e preenche a matriz de entrada A
   if(preencheMatriz(&h_a, n, n) == 0) {
      fprintf(stderr, "Erro de preenchimento da matriz de entrada A\n");
      exit(EXIT_FAILURE);
   }

   //aloca e preenche a matriz de entrada B
   if(preencheMatriz(&h_b, n, n) == 0) {
      fprintf(stderr, "Erro de preenchimento da matriz de entrada B\n");
      exit(EXIT_FAILURE);
   } 
   //aloca a matriz de saida (versao sequencial)
   h_c = (float*) malloc(n_bytes);
   if(h_c==NULL) {
      fprintf(stderr, "Erro de alocacao da matriz de saida\n");
      exit(EXIT_FAILURE);
   }
   //aloca a matriz de saida (versao paralela)
   h_c_par = (float*) malloc(n_bytes);
   if(h_c_par==NULL) {
      fprintf(stderr, "Erro de alocacao da matriz de saida\n");
      exit(EXIT_FAILURE);
   }

   //!!! ------------------------ executa sequencial ---------------------------------- !!!//
   GET_TIME(inicio);
   multMatSeq(h_a, h_b, h_c, n);
   GET_TIME(fim);

   tempo_seq = fim-inicio; // calcula o tempo sequencial em segundos
   //imprime a matriz de saida sequencial
   //printf("\nMatriz de saida C (sequencial):\n");
   //imprimeMatriz(h_c, n, n, stdout);
   

   //!!! ------------------------ executa em paralelo em CUDA -------------------------- !!!//
   GET_TIME(inicio);
   //aloca espaco para as matrizes na GPU
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_a, n_bytes));
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_b, n_bytes));
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_c, n_bytes));

   //copia as matrizes de entrada da CPU para a GPU (host para device)
   CUDA_SAFE_CALL(cudaMemcpy(d_a, h_a, n_bytes, cudaMemcpyHostToDevice));
   CUDA_SAFE_CALL(cudaMemcpy(d_b, h_b, n_bytes, cudaMemcpyHostToDevice));

   //invoca o kernel com blocos de tamanhos fixos
   dim3 threadsBloco = {tam_bloco, tam_bloco, 1};
   dim3 blocosGrade = {n/threadsBloco.x, n/threadsBloco.y, 1};
   
   GET_TIME(fim);
   tempo_par_ini = fim-inicio; // calcula o tempo das inicializacoes paralelo em segundos
   
   printf("kernel com (%d,%d) blocos de (%d,%d) threads\n", blocosGrade.x, blocosGrade.y, threadsBloco.x, threadsBloco.y);

   //dispara o kernel
   CUDA_SAFE_CALL(cudaEventCreate(&start));
   CUDA_SAFE_CALL(cudaEventCreate(&stop));
   CUDA_SAFE_CALL(cudaEventRecord(start));
   multMatPar<<<blocosGrade, threadsBloco>>>(d_a, d_b, d_c, n);
   CUDA_SAFE_CALL(cudaGetLastError());
   CUDA_SAFE_CALL(cudaEventRecord(stop));
   CUDA_SAFE_CALL(cudaEventSynchronize(stop));
   float delta_eventos = 0;
   CUDA_SAFE_CALL(cudaEventElapsedTime(&delta_eventos, start, stop));

   //copia resultado da GPU para a CPU (device para host)
   GET_TIME(inicio);
   CUDA_SAFE_CALL(cudaMemcpy(h_c_par, d_c, n_bytes, cudaMemcpyDeviceToHost))
   GET_TIME(fim);
   tempo_par_fim = fim-inicio; // calcula o tempo das finalizacoes paralelo em segundos
   //imprime a matriz de saida paralela
   //imprimeMatriz(h_c_par, n, n, arqC);
   
   //verifica se o resultado esta correto
   for (int i=0; i<n; i++) {
      for (int j=0; j<n; j++) {
            if (fabs(h_c[i*n+j] - h_c_par[i*n+j]) > 1e-5) {
               fprintf(stderr, "resultado incorreto\n");
               exit(EXIT_FAILURE);
             }
      }
   }
   printf("PASSOU NO TESTE\n");

   //libera a memoria na GPU
   CUDA_SAFE_CALL(cudaFree(d_a));
   CUDA_SAFE_CALL(cudaFree(d_b));
   CUDA_SAFE_CALL(cudaFree(d_c));

   //libera a memoria na CPU
   free(h_a);
   free(h_b);
   free(h_c);
   free(h_c_par);

   //------------------------------- imprime dos tempos de execucao ----------------------//
   printf("Tempo sequencial      = %g seg \n", tempo_seq);
   
   printf("Tempo paralelo kernel = %f seg \n", delta_eventos/1000);
   printf("Tempo paralelo inicio = %f seg \n", tempo_par_ini);
   printf("Tempo paralelo fim    = %f seg \n", tempo_par_fim);
   printf("Tempo paralelo total  = %f seg \n", tempo_par_ini+(delta_eventos/1000)+tempo_par_fim);
   
   printf("\nAceleracao  = %f\n", tempo_seq/(tempo_par_ini+(delta_eventos/1000)+tempo_par_fim));

   return 0;
}

Writing multMatrix.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -o multMatrix multMatrix.cu

In [ ]:
!./multMatrix 2048 16

kernel com (128,128) blocos de (16,16) threads
PASSOU NO TESTE
Tempo sequencial      = 68.2975 seg 
Tempo paralelo kernel = 0.995490 seg 
Tempo paralelo inicio = 0.110163 seg 
Tempo paralelo fim    = 0.013613 seg 
Tempo paralelo total  = 1.119266 seg 

Aceleracao  = 61.019864


In [ ]:
%%writefile multMatrix1.cu
/*
Descricao: multiplicacao de matrizes em paralelo usando GPU
Entrada: dimensao das matrizes e dos blocos de threads
Saida: tempos de execucao
Restricoes: por simplificacao, matrizes quadradas inicializadas para teste.
            A dimensao das matrizes e dos blocos deve ser potencia de 2.  
*/

#include <stdio.h>
//para tomada de tempo
#include <sys/time.h>
//o agumento dee ser double
#define GET_TIME(now) { \
   struct timespec time; \
   clock_gettime(CLOCK_MONOTONIC_RAW, &time); \
   now = time.tv_sec + time.tv_nsec/1000000000.0; \
  }

//para checar erros chamadas Cuda
#define CUDA_SAFE_CALL(call) { \
   cudaError_t err = call;     \
   if(err != cudaSuccess) {    \
      fprintf(stderr,"Erro no arquivo '%s', linha %i: %s.\n",__FILE__, __LINE__,cudaGetErrorString(err)); \
      exit(EXIT_FAILURE); } } 

//funcao para execucao sequencial
void multMatSeq(float *a, float *b, float *c, int n) {
   int i, j, k;
   float soma;
   for(i=0; i<n; i++)
      for(j=0; j<n; j++) {
         soma = 0;
         for(k=0; k<n; k++) {
            soma += a[i*n+k] * b[k*n+j];
         }
         c[i*n+j] = soma;
      }
}

//kernel para execucao paralela em CUDA (!!assume-se que o tamanho do vetor sera sempre potencia de 2!!)
//e que foram disparadas uma thread para cada elemento da matriz de saida
__global__ void multMatPar(float *a, float *b, float *c, int n) {
   //coordenadas globais da thread   
   int i = blockIdx.x * blockDim.x + threadIdx.x;
   int j = blockIdx.y * blockDim.y + threadIdx.y;

   //calcula o elemento C(i,j)
   float valor = 0;
   for (int k = 0; k < n; k++) {
      valor += a[i*n+k] * b[k*n+j];
   }
   //escreve o valor calculado na matriz de saida
   c[i*n+j] = valor;
}

//Kernel de multiplicacao de matrizes com memória compartilhada
__global__ void multMatriz(float *a,float *b, float *c, int n) {
     
//coordenadas globais da thread
int i = blockIdx.x * blockDim.x + threadIdx.x;
int j = blockIdx.y * blockDim.y + threadIdx.y;
    
//coordenadas locais da thread
int i_bloco = threadIdx.x;
int j_bloco = threadIdx.y;


int tam_bloco = 32;
    
//memoria compartilhada para a submatriz de A
__shared__ float Asub[32][32];
//memoria compartilhada para a submatriz de B
__shared__ float Bsub[32][32];
    
//calcula o elemento C(i,j)
    
float valor = 0;

for(int passo=0; passo<n; passo+=tam_bloco) {
  //cada thread carrega um elemento de A e B
  Asub[i_bloco][j_bloco] = a [(i * n) + (passo + j_bloco)];
  Bsub[i_bloco][j_bloco] = b[(passo + i_bloco)*n + j];
  //sincroniza para terminar a copia
  __syncthreads();
  //cada thread computa um elemento
  for (int k = 0; k < tam_bloco; k++) {
  valor += Asub[i_bloco][k] * Bsub[k][j_bloco];
  }
  //sincroniza para terminar a computacão
  __syncthreads();
}
    
//escreve o valor calculado na matriz de saida
c[i*n +j] = valor;
    
}

//funcao que aloca espaco para uma matriz e preenche seus valores
//entrada: matriz de entrada, dimensoes da matriz
//saida: retorna 1 se a matriz foi preenchida com sucesso e 0 caso contrario
int preencheMatriz(float **mat, int linhas, int colunas) {
   int i, j;
   //aloca espaco de memoria para a matriz
   *mat = (float*) malloc(sizeof(float) * linhas * colunas);
   if (mat == NULL) return 0;
   //preenche o vetor
   for (i=0; i<linhas; i++) {
      for (j=0; j<colunas; j++) {
         *((*mat) + (i*colunas+j)) = 1.5;
      }
   }
   return 1;
}

//funcao que imprime uma matriz
//entrada: matriz de entrada, dimensoes da matriz
//saida: matriz impressa na tela
void imprimeMatriz(float *mat, int linhas, int colunas, FILE *arq) {
   int i, j;
   for (i=0; i<linhas; i++) {
      for (j=0; j<colunas; j++) {
         fprintf(arq, "%.1f ", mat[i*colunas+j]);
      }
      fprintf(arq, "\n");
   }
}

//funcao principal
int main(int argc, char** argv) {
   float *h_a, *h_b, *h_c, *h_c_par; //matrizes host
   float *d_a, *d_b, *d_c; //matrizes device
   //para medidas de tempo
   double inicio, fim;
   double tempo_seq, tempo_par_ini, tempo_par_fim;
   cudaEvent_t start, stop;
   //entrada de dados
   //FILE *arqA, *arqB, *arqC; //arquivos dos dados de entrada e saida
   unsigned int n; //dimensao da matriz de entrada
   long int n_bytes; //qtde bytes por matriz
   
   //tamanho dos blocos de threads
   unsigned int tam_bloco;

   //le e valida os parametros de entrada
   if(argc < 3) {
      fprintf(stderr, "Digite: %s <dimensao matrizes (potencia de 2)> <tamanho blocos (potencia de 2, max 32)>.\n", argv[0]);
      exit(EXIT_FAILURE);
   }
   //dimensao das matrizes e tamanho dos blocos
   n = atol(argv[1]);   
   tam_bloco = atol(argv[2]);   

   //calcula o tamanho em bytes das matrizes
   n_bytes = n*n*sizeof(float);

   //aloca e preenche a matriz de entrada A
   if(preencheMatriz(&h_a, n, n) == 0) {
      fprintf(stderr, "Erro de preenchimento da matriz de entrada A\n");
      exit(EXIT_FAILURE);
   }

   //aloca e preenche a matriz de entrada B
   if(preencheMatriz(&h_b, n, n) == 0) {
      fprintf(stderr, "Erro de preenchimento da matriz de entrada B\n");
      exit(EXIT_FAILURE);
   } 
   //aloca a matriz de saida (versao sequencial)
   h_c = (float*) malloc(n_bytes);
   if(h_c==NULL) {
      fprintf(stderr, "Erro de alocacao da matriz de saida\n");
      exit(EXIT_FAILURE);
   }
   //aloca a matriz de saida (versao paralela)
   h_c_par = (float*) malloc(n_bytes);
   if(h_c_par==NULL) {
      fprintf(stderr, "Erro de alocacao da matriz de saida\n");
      exit(EXIT_FAILURE);
   }

   //!!! ------------------------ executa sequencial ---------------------------------- !!!//
   GET_TIME(inicio);
   multMatSeq(h_a, h_b, h_c, n);
   GET_TIME(fim);

   tempo_seq = fim-inicio; // calcula o tempo sequencial em segundos
   //imprime a matriz de saida sequencial
   //printf("\nMatriz de saida C (sequencial):\n");
   //imprimeMatriz(h_c, n, n, stdout);
   

   //!!! ------------------------ executa em paralelo em CUDA -------------------------- !!!//
   GET_TIME(inicio);
   //aloca espaco para as matrizes na GPU
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_a, n_bytes));
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_b, n_bytes));
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_c, n_bytes));

   //copia as matrizes de entrada da CPU para a GPU (host para device)
   CUDA_SAFE_CALL(cudaMemcpy(d_a, h_a, n_bytes, cudaMemcpyHostToDevice));
   CUDA_SAFE_CALL(cudaMemcpy(d_b, h_b, n_bytes, cudaMemcpyHostToDevice));

   //invoca o kernel com blocos de tamanhos fixos
   dim3 threadsBloco = {tam_bloco, tam_bloco, 1};
   dim3 blocosGrade = {n/threadsBloco.x, n/threadsBloco.y, 1};
   
   GET_TIME(fim);
   tempo_par_ini = fim-inicio; // calcula o tempo das inicializacoes paralelo em segundos
   
   printf("kernel com (%d,%d) blocos de (%d,%d) threads\n", blocosGrade.x, blocosGrade.y, threadsBloco.x, threadsBloco.y);

   //dispara o kernel
   CUDA_SAFE_CALL(cudaEventCreate(&start));
   CUDA_SAFE_CALL(cudaEventCreate(&stop));
   CUDA_SAFE_CALL(cudaEventRecord(start));
   //multMatPar<<<blocosGrade, threadsBloco>>>(d_a, d_b, d_c, n);
   multMatriz<<<blocosGrade, threadsBloco>>>(d_a,d_b,d_c,n);
   CUDA_SAFE_CALL(cudaGetLastError());
   CUDA_SAFE_CALL(cudaEventRecord(stop));
   CUDA_SAFE_CALL(cudaEventSynchronize(stop));
   float delta_eventos = 0;
   CUDA_SAFE_CALL(cudaEventElapsedTime(&delta_eventos, start, stop));

   //copia resultado da GPU para a CPU (device para host)
   GET_TIME(inicio);
   CUDA_SAFE_CALL(cudaMemcpy(h_c_par, d_c, n_bytes, cudaMemcpyDeviceToHost))
   GET_TIME(fim);
   tempo_par_fim = fim-inicio; // calcula o tempo das finalizacoes paralelo em segundos
   //imprime a matriz de saida paralela
   //imprimeMatriz(h_c_par, n, n, arqC);
   
   //verifica se o resultado esta correto
   for (int i=0; i<n; i++) {
      for (int j=0; j<n; j++) {
            if (fabs(h_c[i*n+j] - h_c_par[i*n+j]) > 1e-5) {
               fprintf(stderr, "resultado incorreto\n");
               exit(EXIT_FAILURE);
             }
      }
   }
   printf("PASSOU NO TESTE\n");

   //libera a memoria na GPU
   CUDA_SAFE_CALL(cudaFree(d_a));
   CUDA_SAFE_CALL(cudaFree(d_b));
   CUDA_SAFE_CALL(cudaFree(d_c));

   //libera a memoria na CPU
   free(h_a);
   free(h_b);
   free(h_c);
   free(h_c_par);

   //------------------------------- imprime dos tempos de execucao ----------------------//
   printf("Tempo sequencial      = %g seg \n", tempo_seq);
   
   printf("Tempo paralelo kernel = %f seg \n", delta_eventos/1000);
   printf("Tempo paralelo inicio = %f seg \n", tempo_par_ini);
   printf("Tempo paralelo fim    = %f seg \n", tempo_par_fim);
   printf("Tempo paralelo total  = %f seg \n", tempo_par_ini+(delta_eventos/1000)+tempo_par_fim);
   
   printf("\nAceleracao  = %f\n", tempo_seq/(tempo_par_ini+(delta_eventos/1000)+tempo_par_fim));

   return 0;
}

Writing multMatrix1.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -o multMatrix1 multMatrix1.cu

In [ ]:
!./multMatrix1 2048 8

kernel com (256,256) blocos de (8,8) threads
PASSOU NO TESTE
Tempo sequencial      = 70.9398 seg 
Tempo paralelo kernel = 0.159076 seg 
Tempo paralelo inicio = 0.110088 seg 
Tempo paralelo fim    = 0.014059 seg 
Tempo paralelo total  = 0.283223 seg 

Aceleracao  = 250.473484


In [ ]:
%%writefile multMatrixDinam.cu
/*
Descricao: multiplicacao de matrizes em paralelo usando GPU
Entrada: dimensao das matrizes e dos blocos de threads
Saida: tempos de execucao
Restricoes: por simplificacao, matrizes quadradas inicializadas para teste.
            A dimensao das matrizes e dos blocos deve ser potencia de 2.  
*/

#include <stdio.h>
//para tomada de tempo
#include <sys/time.h>
//o agumento dee ser double
#define GET_TIME(now) { \
   struct timespec time; \
   clock_gettime(CLOCK_MONOTONIC_RAW, &time); \
   now = time.tv_sec + time.tv_nsec/1000000000.0; \
  }

//para checar erros chamadas Cuda
#define CUDA_SAFE_CALL(call) { \
   cudaError_t err = call;     \
   if(err != cudaSuccess) {    \
      fprintf(stderr,"Erro no arquivo '%s', linha %i: %s.\n",__FILE__, __LINE__,cudaGetErrorString(err)); \
      exit(EXIT_FAILURE); } } 

//funcao para execucao sequencial
void multMatSeq(float *a, float *b, float *c, int n) {
   int i, j, k;
   float soma;
   for(i=0; i<n; i++)
      for(j=0; j<n; j++) {
         soma = 0;
         for(k=0; k<n; k++) {
            soma += a[i*n+k] * b[k*n+j];
         }
         c[i*n+j] = soma;
      }
}

//kernel para execucao paralela em CUDA (!!assume-se que o tamanho do vetor sera sempre potencia de 2!!)
//e que foram disparadas uma thread para cada elemento da matriz de saida
__global__ void multMatPar(float *a, float *b, float *c, int n) {
   //coordenadas globais da thread   
   int i = blockIdx.x * blockDim.x + threadIdx.x;
   int j = blockIdx.y * blockDim.y + threadIdx.y;

   //calcula o elemento C(i,j)
   float valor = 0;
   for (int k = 0; k < n; k++) {
      valor += a[i*n+k] * b[k*n+j];
   }
   //escreve o valor calculado na matriz de saida
   c[i*n+j] = valor;
}


//Kernel de multiplicacao de matrizes com memória compartilhada
__global__ void multMatriz(float *a,float *b, float *c, int n) {
     
//coordenadas globais da thread
int i = blockIdx.x * blockDim.x + threadIdx.x;
int j = blockIdx.y * blockDim.y + threadIdx.y;
    
//coordenadas locais da thread
int i_bloco = threadIdx.x;
int j_bloco = threadIdx.y;


int tam_bloco = 8;
    
//memoria compartilhada para a submatriz de A
__shared__ float Asub[8][8];
//memoria compartilhada para a submatriz de B
__shared__ float Bsub[8][8];
    
//calcula o elemento C(i,j)
    
float valor = 0;

for(int passo=0; passo<n; passo+=tam_bloco) {
  //cada thread carrega um elemento de A e B
  Asub[i_bloco][j_bloco] = a [(i * n) + (passo + j_bloco)];
  Bsub[i_bloco][j_bloco] = b[(passo + i_bloco)*n + j];
  //sincroniza para terminar a copia
  __syncthreads();
  //cada thread computa um elemento
  for (int k = 0; k < tam_bloco; k++) {
  valor += Asub[i_bloco][k] * Bsub[k][j_bloco];
  }
  //sincroniza para terminar a computacão
  __syncthreads();
}
    
//escreve o valor calculado na matriz de saida
c[i*n +j] = valor;
    
}

//Kernel de multiplicacao de matrizes com memória compartilhada dinamica
__global__ void multMatrizDinam(float *a,float *b, float *c, int n) {
     
//coordenadas globais da thread
int i = blockIdx.x * blockDim.x + threadIdx.x;
int j = blockIdx.y * blockDim.y + threadIdx.y;
    
//coordenadas locais da thread
int i_bloco = threadIdx.x;
int j_bloco = threadIdx.y;


extern __shared__ float mat_sub[];
    
//memoria compartilhada para a submatriz de A    
float* Asub = (float*) mat_sub;
//memoria compartilhada para a submatriz de B
float* Bsub= (float*) &Asub[blockDim.x*blockDim.y];
    
//calcula o elemento C(i,j)
    
float valor = 0;

for(int passo=0; passo<n; passo+=blockDim.y) {
  //cada thread carrega um elemento de A e B
  Asub[i_bloco* blockDim.y + j_bloco] = a [(i * n) + (passo + j_bloco)];
  Bsub[i_bloco*blockDim.y + j_bloco] = b[(passo + i_bloco)*n + j];
  //sincroniza para terminar a copia
  __syncthreads();
  //cada thread computa um elemento
  for (int k = 0; k < blockDim.y; k++) {
  valor += Asub[i_bloco*blockDim.y + k] * Bsub[k*blockDim.y + j_bloco];
  }
  //sincroniza para terminar a computacão
  __syncthreads();
}
    
//escreve o valor calculado na matriz de saida
c[i*n +j] = valor;
    
}

//funcao que aloca espaco para uma matriz e preenche seus valores
//entrada: matriz de entrada, dimensoes da matriz
//saida: retorna 1 se a matriz foi preenchida com sucesso e 0 caso contrario
int preencheMatriz(float **mat, int linhas, int colunas) {
   int i, j;
   //aloca espaco de memoria para a matriz
   *mat = (float*) malloc(sizeof(float) * linhas * colunas);
   if (mat == NULL) return 0;
   //preenche o vetor
   for (i=0; i<linhas; i++) {
      for (j=0; j<colunas; j++) {
         *((*mat) + (i*colunas+j)) = 1.5;
      }
   }
   return 1;
}

//funcao que imprime uma matriz
//entrada: matriz de entrada, dimensoes da matriz
//saida: matriz impressa na tela
void imprimeMatriz(float *mat, int linhas, int colunas, FILE *arq) {
   int i, j;
   for (i=0; i<linhas; i++) {
      for (j=0; j<colunas; j++) {
         fprintf(arq, "%.1f ", mat[i*colunas+j]);
      }
      fprintf(arq, "\n");
   }
}

//funcao principal
int main(int argc, char** argv) {
   float *h_a, *h_b, *h_c, *h_c_par; //matrizes host
   float *d_a, *d_b, *d_c; //matrizes device
   //para medidas de tempo
   double inicio, fim;
   double tempo_seq, tempo_par_ini, tempo_par_fim;
   cudaEvent_t start, stop;
   //entrada de dados
   //FILE *arqA, *arqB, *arqC; //arquivos dos dados de entrada e saida
   unsigned int n; //dimensao da matriz de entrada
   long int n_bytes; //qtde bytes por matriz
   
   //tamanho dos blocos de threads
   unsigned int tam_bloco;

   //le e valida os parametros de entrada
   if(argc < 3) {
      fprintf(stderr, "Digite: %s <dimensao matrizes (potencia de 2)> <tamanho blocos (potencia de 2, max 32)>.\n", argv[0]);
      exit(EXIT_FAILURE);
   }
   //dimensao das matrizes e tamanho dos blocos
   n = atol(argv[1]);   
   tam_bloco = atol(argv[2]);   

   //calcula o tamanho em bytes das matrizes
   n_bytes = n*n*sizeof(float);

   //aloca e preenche a matriz de entrada A
   if(preencheMatriz(&h_a, n, n) == 0) {
      fprintf(stderr, "Erro de preenchimento da matriz de entrada A\n");
      exit(EXIT_FAILURE);
   }

   //aloca e preenche a matriz de entrada B
   if(preencheMatriz(&h_b, n, n) == 0) {
      fprintf(stderr, "Erro de preenchimento da matriz de entrada B\n");
      exit(EXIT_FAILURE);
   } 
   //aloca a matriz de saida (versao sequencial)
   h_c = (float*) malloc(n_bytes);
   if(h_c==NULL) {
      fprintf(stderr, "Erro de alocacao da matriz de saida\n");
      exit(EXIT_FAILURE);
   }
   //aloca a matriz de saida (versao paralela)
   h_c_par = (float*) malloc(n_bytes);
   if(h_c_par==NULL) {
      fprintf(stderr, "Erro de alocacao da matriz de saida\n");
      exit(EXIT_FAILURE);
   }

   //!!! ------------------------ executa sequencial ---------------------------------- !!!//
   GET_TIME(inicio);
   //multMatSeq(h_a, h_b, h_c, n);
   GET_TIME(fim);

   tempo_seq = fim-inicio; // calcula o tempo sequencial em segundos
   //imprime a matriz de saida sequencial
   //printf("\nMatriz de saida C (sequencial):\n");
   //imprimeMatriz(h_c, n, n, stdout);
   

   //!!! ------------------------ executa em paralelo em CUDA -------------------------- !!!//
   GET_TIME(inicio);
   //aloca espaco para as matrizes na GPU
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_a, n_bytes));
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_b, n_bytes));
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_c, n_bytes));

   //copia as matrizes de entrada da CPU para a GPU (host para device)
   CUDA_SAFE_CALL(cudaMemcpy(d_a, h_a, n_bytes, cudaMemcpyHostToDevice));
   CUDA_SAFE_CALL(cudaMemcpy(d_b, h_b, n_bytes, cudaMemcpyHostToDevice));

   //invoca o kernel com blocos de tamanhos fixos
   dim3 threadsBloco = {tam_bloco, tam_bloco, 1};
   dim3 blocosGrade = {n/threadsBloco.x, n/threadsBloco.y, 1};
   
   GET_TIME(fim);
   tempo_par_ini = fim-inicio; // calcula o tempo das inicializacoes paralelo em segundos
   
   printf("kernel com (%d,%d) blocos de (%d,%d) threads\n", blocosGrade.x, blocosGrade.y, threadsBloco.x, threadsBloco.y);
    
   int tamMemCompartilhada = tam_bloco*tam_bloco*sizeof(float)*2; 
    
   //dispara o kernel
   CUDA_SAFE_CALL(cudaEventCreate(&start));
   CUDA_SAFE_CALL(cudaEventCreate(&stop));
   CUDA_SAFE_CALL(cudaEventRecord(start));
   multMatPar<<<blocosGrade, threadsBloco>>>(d_a, d_b, d_c, n);
   //multMatriz<<<blocosGrade, threadsBloco>>>(d_a, d_b, d_c, n);
   //multMatrizDinam<<<blocosGrade, threadsBloco,tamMemCompartilhada>>>(d_a,d_b,d_c,n);
   CUDA_SAFE_CALL(cudaGetLastError());
   CUDA_SAFE_CALL(cudaEventRecord(stop));
   CUDA_SAFE_CALL(cudaEventSynchronize(stop));
   float delta_eventos = 0;
   CUDA_SAFE_CALL(cudaEventElapsedTime(&delta_eventos, start, stop));

   //copia resultado da GPU para a CPU (device para host)
   GET_TIME(inicio);
   CUDA_SAFE_CALL(cudaMemcpy(h_c_par, d_c, n_bytes, cudaMemcpyDeviceToHost))
   GET_TIME(fim);
   tempo_par_fim = fim-inicio; // calcula o tempo das finalizacoes paralelo em segundos
   //imprime a matriz de saida paralela
   //imprimeMatriz(h_c_par, n, n, arqC);
   
   //verifica se o resultado esta correto
   /*for (int i=0; i<n; i++) {
      for (int j=0; j<n; j++) {
            if (fabs(h_c[i*n+j] - h_c_par[i*n+j]) > 1e-5) {
               fprintf(stderr, "resultado incorreto\n");
               exit(EXIT_FAILURE);
             }
      }
   }*/
   
   printf("PASSOU NO TESTE\n");

   //libera a memoria na GPU
   CUDA_SAFE_CALL(cudaFree(d_a));
   CUDA_SAFE_CALL(cudaFree(d_b));
   CUDA_SAFE_CALL(cudaFree(d_c));

   //libera a memoria na CPU
   free(h_a);
   free(h_b);
   free(h_c);
   free(h_c_par);

   //------------------------------- imprime dos tempos de execucao ----------------------//
   printf("Tempo sequencial      = %g seg \n", tempo_seq);
   
   printf("Tempo paralelo kernel = %f seg \n", delta_eventos/1000);
   printf("Tempo paralelo inicio = %f seg \n", tempo_par_ini);
   printf("Tempo paralelo fim    = %f seg \n", tempo_par_fim);
   printf("Tempo paralelo total  = %f seg \n", tempo_par_ini+(delta_eventos/1000)+tempo_par_fim);
   
   printf("\nAceleracao  = %f\n", 67.8144/(tempo_par_ini+(delta_eventos/1000)+tempo_par_fim));

   return 0;
}

Writing multMatrixDinam.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -o multMatrixDinam multMatrixDinam.cu

multMatrixDinam.cu(254): warning: variable "tamMemCompartilhada" was declared but never referenced



In [ ]:
!./multMatrixDinam 2048 8

kernel com (256,256) blocos de (8,8) threads
PASSOU NO TESTE
Tempo sequencial      = 1.394e-06 seg 
Tempo paralelo kernel = 0.594067 seg 
Tempo paralelo inicio = 0.175018 seg 
Tempo paralelo fim    = 0.012971 seg 
Tempo paralelo total  = 0.782056 seg 

Aceleracao  = 86.712983


In [ ]:
%%writefile multMatrixGeneric.cu
/*
Descricao: multiplicacao de matrizes em paralelo usando GPU
Entrada: dimensao das matrizes e dos blocos de threads
Saida: tempos de execucao
Restricoes: por simplificacao, matrizes quadradas inicializadas para teste.
            A dimensao das matrizes e dos blocos deve ser potencia de 2.  
*/

#include <stdio.h>
//para tomada de tempo
#include <sys/time.h>
//o agumento dee ser double
#define BLOCK_SIZE 16
#define GET_TIME(now) { \
   struct timespec time; \
   clock_gettime(CLOCK_MONOTONIC_RAW, &time); \
   now = time.tv_sec + time.tv_nsec/1000000000.0; \
  }



//para checar erros chamadas Cuda
#define CUDA_SAFE_CALL(call) { \
   cudaError_t err = call;     \
   if(err != cudaSuccess) {    \
      fprintf(stderr,"Erro no arquivo '%s', linha %i: %s.\n",__FILE__, __LINE__,cudaGetErrorString(err)); \
      exit(EXIT_FAILURE); } } 

//funcao para execucao sequencial
void multMatSeq(float *a, float *b, float *c, int rowA,int colA,int colB) {
   int i, j, k;
   float soma;
   for(i=0; i<rowA; i++)
      for(j=0; j<colB; j++) {
         soma = 0;
         for(k=0; k<colA; k++) {
            soma += a[i*colA+k] * b[k*colB+j];
         }
         c[i*colB+j] = soma;
      }
}


//Kernel de multiplicacao de matrizes genéricas com memória compartilhada
__global__ void multMatriz(float *a,float *b, float *c, int colunasA, int colunasB) {
     
//coordenadas globais da thread
int i = blockIdx.x * blockDim.x + threadIdx.x;
int j = blockIdx.y * blockDim.y + threadIdx.y;
    
//coordenadas locais da thread
int i_bloco = threadIdx.x;
int j_bloco = threadIdx.y;


int tam_bloco = BLOCK_SIZE;
    
//memoria compartilhada para a submatriz de A
__shared__ float Asub[BLOCK_SIZE][BLOCK_SIZE];
//memoria compartilhada para a submatriz de B
__shared__ float Bsub[BLOCK_SIZE][BLOCK_SIZE];
    
//calcula o elemento C(i,j)
    
float valor = 0;

for(int passo=0; passo<colunasA; passo+=tam_bloco) {
  //cada thread carrega um elemento de A e B
  Asub[i_bloco][j_bloco] = a [(i * colunasA) + (passo + j_bloco)];
  Bsub[i_bloco][j_bloco] = b[(passo + i_bloco)*colunasB + j];
  //sincroniza para terminar a copia
  __syncthreads();
  //cada thread computa um elemento
  for (int k = 0; k < tam_bloco; k++) {
  valor += Asub[i_bloco][k] * Bsub[k][j_bloco];
  }
  //sincroniza para terminar a computacão
  __syncthreads();
}
    
//escreve o valor calculado na matriz de saida
c[i*colunasB +j] = valor;
    
}


//funcao que aloca espaco para uma matriz e preenche seus valores
//entrada: matriz de entrada, dimensoes da matriz
//saida: retorna 1 se a matriz foi preenchida com sucesso e 0 caso contrario
int preencheMatriz(float **mat, int linhas, int colunas) {
   int i, j;
   //aloca espaco de memoria para a matriz
   *mat = (float*) malloc(sizeof(float) * linhas * colunas);
   if (mat == NULL) return 0;
   //preenche o vetor
   for (i=0; i<linhas; i++) {
      for (j=0; j<colunas; j++) {
         *((*mat) + (i*colunas+j)) = 1.5;
      }
   }
   return 1;
}

//funcao que imprime uma matriz
//entrada: matriz de entrada, dimensoes da matriz
//saida: matriz impressa na tela
void imprimeMatriz(float *mat, int linhas, int colunas, FILE *arq) {
   int i, j;
   for (i=0; i<linhas; i++) {
       
      for (j=0; j<colunas; j++) {
         fprintf(arq, "%.1f ", mat[i*colunas+j]);
      }
      fprintf(arq, "\n");
   }
}

//funcao principal
int main(int argc, char** argv) {
   float *h_a, *h_b, *h_c, *h_c_par; //matrizes host
   float *d_a, *d_b, *d_c; //matrizes device
   //para medidas de tempo
   double inicio, fim;
   double tempo_seq, tempo_par_ini, tempo_par_fim;
   cudaEvent_t start, stop;
   long int n_bytes; //qtde bytes por matriz
   unsigned int tam_bloco = BLOCK_SIZE;
  
   int real_linhasA;
	 int real_linhasB;
	 int real_colunasA;
	 int real_colunasB;
    
   
    
	if(argc != 5){
		printf("Modo de uso: %s <Linhas Matriz A> <Colunas Matriz A> <Linhas Matriz B> <Colunas Matriz B>\n", argv[0]);
		exit(-1);
	}else{
		real_linhasA  = atoi(argv[1]);
		real_colunasA = atoi(argv[2]);
		real_linhasB  = atoi(argv[3]);
		real_colunasB = atoi(argv[4]);

		if(real_colunasA != real_linhasB){
			printf("Erro: Colunas A != Linhas B\n");
			exit(-1);
		}
  }
   int linhasA;
	 int linhasB;
	 int colunasA;
   int colunasB;
  
 //Completa as linhas e colunas para serem múltiplas de block_size
    linhasA  = real_linhasA  +BLOCK_SIZE - (real_linhasA  % BLOCK_SIZE);
    colunasA = real_colunasA + BLOCK_SIZE- (real_colunasA % BLOCK_SIZE);

	  linhasB  = real_linhasB  + BLOCK_SIZE - (real_linhasB  % BLOCK_SIZE);
	  colunasB = real_colunasB + BLOCK_SIZE - (real_colunasB % BLOCK_SIZE);

    n_bytes = linhasA * colunasB * sizeof(float);
    
   /*if(preencheMatriz(&h_a, linhasA, colunasA) == 0) {
      fprintf(stderr, "Erro de preenchimento da matriz de entrada A\n");
      exit(EXIT_FAILURE);
   }

   //aloca e preenche a matriz de entrada B
   if(preencheMatriz(&h_b, linhasB, colunasB) == 0) {
      fprintf(stderr, "Erro de preenchimento da matriz de entrada B\n");
      exit(EXIT_FAILURE);
   }
    */
   //aloca a matriz de saida (versao sequencial)
   h_c = (float*) malloc(n_bytes);
   if(h_c==NULL) {
      fprintf(stderr, "Erro de alocacao da matriz de saida\n");
      exit(EXIT_FAILURE);
   }
   //aloca a matriz de saida (versao paralela)
   h_c_par = (float*) malloc(n_bytes);
   if(h_c_par==NULL) {
      fprintf(stderr, "Erro de alocacao da matriz de saida\n");
      exit(EXIT_FAILURE);
   }
    
    
    h_a = (float *) malloc(linhasA * colunasA * sizeof(float));
    h_b = (float *) malloc(linhasB * colunasB * sizeof(float));
    
    //Inicializa os vetores no host
	for(int i = 0; i < linhasA * colunasA; i++){

		if(i%colunasA < real_linhasA || i%linhasA < real_colunasA){
			h_a[i] = (double) 1.0;
		}else{
			//Preenche com zeros o que completamos anteriormente
			h_a[i] = 0;
		}	
	}

	//Inicializa os vetores no host
	for(int i = 0; i < linhasB * colunasB; i++){
		if(i%colunasB < real_linhasB || i%linhasB < real_colunasB){
			h_b[i] = (double) 1.0;
		}else{
			//Preenche com zeros o que completamos anteriormente
			h_b[i] = 0;
		}	
	} 

    
   //!!! ------------------------ executa sequencial ---------------------------------- !!!//
   GET_TIME(inicio);
   multMatSeq(h_a, h_b, h_c, linhasA,colunasA,colunasB);
   GET_TIME(fim);

   tempo_seq = fim-inicio; // calcula o tempo sequencial em segundos
   

   //!!! ------------------------ executa em paralelo em CUDA -------------------------- !!!//
   GET_TIME(inicio);
   //aloca espaco para as matrizes na GPU
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_a, linhasA * colunasA * sizeof(float)));
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_b, linhasB * colunasB * sizeof(float)));
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_c, n_bytes));

   //copia as matrizes de entrada da CPU para a GPU (host para device)
   CUDA_SAFE_CALL(cudaMemcpy(d_a, h_a, linhasA * colunasA * sizeof(float), cudaMemcpyHostToDevice));
   CUDA_SAFE_CALL(cudaMemcpy(d_b, h_b, linhasB * colunasB * sizeof(float) , cudaMemcpyHostToDevice));

   //invoca o kernel com blocos de tamanhos fixos
   dim3 threadsBloco = {tam_bloco, tam_bloco};
   dim3 blocosGrade = {linhasA/threadsBloco.x, colunasB/threadsBloco.y};
   
   GET_TIME(fim);
   tempo_par_ini = fim-inicio; // calcula o tempo das inicializacoes paralelo em segundos
   
   printf("kernel com (%d,%d) blocos de (%d,%d) threads\n", blocosGrade.x, blocosGrade.y, threadsBloco.x, threadsBloco.y);
 
    
   //dispara o kernel
   CUDA_SAFE_CALL(cudaEventCreate(&start));
   CUDA_SAFE_CALL(cudaEventCreate(&stop));
   CUDA_SAFE_CALL(cudaEventRecord(start));
   multMatriz<<<blocosGrade, threadsBloco>>>(d_a, d_b, d_c, colunasA,colunasB);
   CUDA_SAFE_CALL(cudaGetLastError());
   CUDA_SAFE_CALL(cudaEventRecord(stop));
   CUDA_SAFE_CALL(cudaEventSynchronize(stop));
   float delta_eventos = 0;
   CUDA_SAFE_CALL(cudaEventElapsedTime(&delta_eventos, start, stop));

   //copia resultado da GPU para a CPU (device para host)
   GET_TIME(inicio);
   CUDA_SAFE_CALL(cudaMemcpy(h_c_par, d_c, n_bytes, cudaMemcpyDeviceToHost))
   GET_TIME(fim);
   tempo_par_fim = fim-inicio; // calcula o tempo das finalizacoes paralelo em segundos
   
   //verifica se o resultado esta correto
   /*for (int i=0; i<linhasA; i++) {
      for (int j=0; j<colunasB; j++) {
            if (fabs(h_c[i*linhasA+j] - h_c_par[i*linhasA+j]) > 1e-5) {
                printf("%f", h_c[i*linhasA+j]);
                printf("- %f", h_c_par[i*linhasA+j]);
               fprintf(stderr, "resultado incorreto\n");
               exit(EXIT_FAILURE);
             }
      }
   }*/
   
   /*
   printf("%d ", linhasA);
   printf("%d - ", colunasB);
   printf("\nMatriz de saida C (sequencial):\n");
   imprimeMatriz(h_c, linhasA , colunasB, stdout);
    
   printf("\nMatriz de saida C (paralelo):\n");
   //imprimeMatriz(h_c_par, linhasA , colunasB, stdout); */
    
   for(int i = 0; i < linhasA * colunasB; i++){
      if(fabs(h_c[i] - h_c_par[i]) > 0.1){
     
          
          printf("%f ", h_c[i]);
          printf("- %f", h_c_par[i]);
          printf(" -- %d",i);
        fprintf(stderr, "resultado incorreto\n");
        exit(EXIT_FAILURE);
      }
   }
   
   printf("PASSOU NO TESTE\n");

   //libera a memoria na GPU
   CUDA_SAFE_CALL(cudaFree(d_a));
   CUDA_SAFE_CALL(cudaFree(d_b));
   CUDA_SAFE_CALL(cudaFree(d_c));

   //libera a memoria na CPU
   free(h_a);
   free(h_b);
   free(h_c);
   free(h_c_par);

   //------------------------------- imprime dos tempos de execucao ----------------------//
   printf("Tempo sequencial      = %g seg \n", tempo_seq);
   
   printf("Tempo paralelo kernel = %f seg \n", delta_eventos/1000);
   printf("Tempo paralelo inicio = %f seg \n", tempo_par_ini);
   printf("Tempo paralelo fim    = %f seg \n", tempo_par_fim);
   printf("Tempo paralelo total  = %f seg \n", tempo_par_ini+(delta_eventos/1000)+tempo_par_fim);
   
   printf("\nAceleracao  = %f\n", tempo_seq/(tempo_par_ini+(delta_eventos/1000)+tempo_par_fim));

   return 0;
}

Writing multMatrixGeneric.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -o multMatrixGeneric multMatrixGeneric.cu

In [ ]:
!./multMatrixGeneric 1024 1024 1024 1024

kernel com (65,65) blocos de (16,16) threads
PASSOU NO TESTE
Tempo sequencial      = 4.68477 seg 
Tempo paralelo kernel = 0.039869 seg 
Tempo paralelo inicio = 0.097007 seg 
Tempo paralelo fim    = 0.003480 seg 
Tempo paralelo total  = 0.140356 seg 

Aceleracao  = 33.377832


In [ ]:
%%writefile lu.cu
/*
Descricao: multiplicacao de matrizes em paralelo usando GPU
Entrada: dimensao das matrizes e dos blocos de threads
Saida: tempos de execucao
Restricoes: por simplificacao, matrizes quadradas inicializadas para teste.
            A dimensao das matrizes e dos blocos deve ser potencia de 2.  
*/

#include <stdio.h>
//para tomada de tempo
#include <sys/time.h>
//o agumento dee ser double
#define GET_TIME(now) { \
   struct timespec time; \
   clock_gettime(CLOCK_MONOTONIC_RAW, &time); \
   now = time.tv_sec + time.tv_nsec/1000000000.0; \
  }

//para checar erros chamadas Cuda
#define CUDA_SAFE_CALL(call) { \
   cudaError_t err = call;     \
   if(err != cudaSuccess) {    \
      fprintf(stderr,"Erro no arquivo '%s', linha %i: %s.\n",__FILE__, __LINE__,cudaGetErrorString(err)); \
      exit(EXIT_FAILURE); } } 

//funcao para execucao sequencial
void LU(float *a, float *l, float *u, int n) {
    for (int i=0; i<n-1; i++) {
        for (int j=i+1; j<n; j++) {
            l[j*n+i] = a[j*n+i] / a[i*n+i];
        }
        for (int j=i+1; j<n; j++) {
            for (int k=i+1; k<n; k++) {
                a[j*n+k] -= a[j*n+i] * a[i*n+k];
            }
        }
    }
}
 
//kernel para execucao paralela em CUDA (!!assume-se que o tamanho do vetor sera sempre potencia de 2!!)
//e que foram disparadas uma thread para cada elemento da matriz de saida
__global__ void multMatPar(float *a, float *b, float *c, int n) {
   //coordenadas globais da thread   
   int i = blockIdx.x * blockDim.x + threadIdx.x;
   int j = blockIdx.y * blockDim.y + threadIdx.y;

   //calcula o elemento C(i,j)
   float valor = 0;
   for (int k = 0; k < n; k++) {
      valor += a[i*n+k] * b[k*n+j];
   }
   //escreve o valor calculado na matriz de saida
   c[i*n+j] = valor;
}

//funcao que aloca espaco para uma matriz e preenche seus valores
//entrada: matriz de entrada, dimensoes da matriz
//saida: retorna 1 se a matriz foi preenchida com sucesso e 0 caso contrario
int preencheMatriz(float **mat, int linhas, int colunas) {
   int i, j;
   //aloca espaco de memoria para a matriz
   *mat = (float*) malloc(sizeof(float) * linhas * colunas);
   if (mat == NULL) return 0;
   //preenche o vetor
   for (i=0; i<linhas; i++) {
      for (j=0; j<colunas; j++) {
         *((*mat) + (i*colunas+j)) = 2.0;
      }
   }
   return 1;
}

//funcao que imprime uma matriz
//entrada: matriz de entrada, dimensoes da matriz
//saida: matriz impressa na tela
void imprimeMatriz(float *mat, int linhas, int colunas, FILE *arq) {
   int i, j;
   for (i=0; i<linhas; i++) {
      for (j=0; j<colunas; j++) {
         fprintf(arq, "%.1f ", mat[i*colunas+j]);
      }
      fprintf(arq, "\n");
   }
}

//funcao principal
int main(int argc, char** argv) {
   float *h_a, *h_b, *h_c, *h_c_par; //matrizes host
   float *d_a, *d_b, *d_c; //matrizes device
   //para medidas de tempo
   double inicio, fim;
   double tempo_seq, tempo_par_ini, tempo_par_fim;
   cudaEvent_t start, stop;
   //entrada de dados
   //FILE *arqA, *arqB, *arqC; //arquivos dos dados de entrada e saida
   unsigned int n; //dimensao da matriz de entrada
   long int n_bytes; //qtde bytes por matriz
   
   //tamanho dos blocos de threads
   unsigned int tam_bloco;

   //le e valida os parametros de entrada
   if(argc < 3) {
      fprintf(stderr, "Digite: %s <dimensao matrizes (potencia de 2)> <tamanho blocos (potencia de 2, max 32)>.\n", argv[0]);
      exit(EXIT_FAILURE);
   }
   //dimensao das matrizes e tamanho dos blocos
   n = 4;   
   tam_bloco = atol(argv[2]);   

   //calcula o tamanho em bytes das matrizes
   n_bytes = n*n*sizeof(float);

   //aloca e preenche a matriz de entrada A
   if(preencheMatriz(&h_a, n, n) == 0) {
      fprintf(stderr, "Erro de preenchimento da matriz de entrada A\n");
      exit(EXIT_FAILURE);
   }

   //aloca e preenche a matriz de entrada B
   if(preencheMatriz(&h_b, n, n) == 0) {
      fprintf(stderr, "Erro de preenchimento da matriz de entrada B\n");
      exit(EXIT_FAILURE);
   } 
   //aloca a matriz de saida (versao sequencial)
   h_c = (float*) malloc(n_bytes);
   if(h_c==NULL) {
      fprintf(stderr, "Erro de alocacao da matriz de saida\n");
      exit(EXIT_FAILURE);
   }
   //aloca a matriz de saida (versao paralela)
   h_c_par = (float*) malloc(n_bytes);
   if(h_c_par==NULL) {
      fprintf(stderr, "Erro de alocacao da matriz de saida\n");
      exit(EXIT_FAILURE);
   }

   //!!! ------------------------ executa sequencial ---------------------------------- !!!//
   GET_TIME(inicio);
   LU(h_a, n);
   
   GET_TIME(fim);

   tempo_seq = fim-inicio; // calcula o tempo sequencial em segundos
   //imprime a matriz de saida sequencial
   printf("\nMatriz de saida C (sequencial):\n");
   imprimeMatriz(h_a, n, n, stdout);
   

   //!!! ------------------------ executa em paralelo em CUDA -------------------------- !!!//
   GET_TIME(inicio);
   //aloca espaco para as matrizes na GPU
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_a, n_bytes));
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_b, n_bytes));
   CUDA_SAFE_CALL(cudaMalloc((void**) &d_c, n_bytes));

   //copia as matrizes de entrada da CPU para a GPU (host para device)
   CUDA_SAFE_CALL(cudaMemcpy(d_a, h_a, n_bytes, cudaMemcpyHostToDevice));
   CUDA_SAFE_CALL(cudaMemcpy(d_b, h_b, n_bytes, cudaMemcpyHostToDevice));

   //invoca o kernel com blocos de tamanhos fixos
   dim3 threadsBloco = {tam_bloco, tam_bloco, 1};
   dim3 blocosGrade = {n/threadsBloco.x, n/threadsBloco.y, 1};
   
   GET_TIME(fim);
   tempo_par_ini = fim-inicio; // calcula o tempo das inicializacoes paralelo em segundos
   
   printf("kernel com (%d,%d) blocos de (%d,%d) threads\n", blocosGrade.x, blocosGrade.y, threadsBloco.x, threadsBloco.y);

   //dispara o kernel
   CUDA_SAFE_CALL(cudaEventCreate(&start));
   CUDA_SAFE_CALL(cudaEventCreate(&stop));
   CUDA_SAFE_CALL(cudaEventRecord(start));
   multMatPar<<<blocosGrade, threadsBloco>>>(d_a, d_b, d_c, n);
   CUDA_SAFE_CALL(cudaGetLastError());
   CUDA_SAFE_CALL(cudaEventRecord(stop));
   CUDA_SAFE_CALL(cudaEventSynchronize(stop));
   float delta_eventos = 0;
   CUDA_SAFE_CALL(cudaEventElapsedTime(&delta_eventos, start, stop));

   //copia resultado da GPU para a CPU (device para host)
   GET_TIME(inicio);
   CUDA_SAFE_CALL(cudaMemcpy(h_c_par, d_c, n_bytes, cudaMemcpyDeviceToHost))
   GET_TIME(fim);
   tempo_par_fim = fim-inicio; // calcula o tempo das finalizacoes paralelo em segundos
   //imprime a matriz de saida paralela
   //imprimeMatriz(h_c_par, n, n, arqC);
   
   //verifica se o resultado esta correto
   for (int i=0; i<n; i++) {
      for (int j=0; j<n; j++) {
            if (fabs(h_c[i*n+j] - h_c_par[i*n+j]) > 1e-5) {
               fprintf(stderr, "resultado incorreto\n");
               exit(EXIT_FAILURE);
             }
      }
   }
   printf("PASSOU NO TESTE\n");

   //libera a memoria na GPU
   CUDA_SAFE_CALL(cudaFree(d_a));
   CUDA_SAFE_CALL(cudaFree(d_b));
   CUDA_SAFE_CALL(cudaFree(d_c));

   //libera a memoria na CPU
   free(h_a);
   free(h_b);
   free(h_c);
   free(h_c_par);

   //------------------------------- imprime dos tempos de execucao ----------------------//
   printf("Tempo sequencial      = %g seg \n", tempo_seq);
   
   printf("Tempo paralelo kernel = %f seg \n", delta_eventos/1000);
   printf("Tempo paralelo inicio = %f seg \n", tempo_par_ini);
   printf("Tempo paralelo fim    = %f seg \n", tempo_par_fim);
   printf("Tempo paralelo total  = %f seg \n", tempo_par_ini+(delta_eventos/1000)+tempo_par_fim);
   
   printf("\nAceleracao  = %f\n", tempo_seq/(tempo_par_ini+(delta_eventos/1000)+tempo_par_fim));

   return 0;
}

Overwriting lu.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -o lu lu.cu

In [ ]:
!./lu 1024 16


Matriz de saida C (sequencial):
2.0 2.0 2.0 2.0 
1.0 2.0 2.0 2.0 
1.0 1.0 2.0 2.0 
1.0 1.0 1.0 2.0 
kernel com (0,0) blocos de (16,16) threads
Erro no arquivo 'lu.cu', linha 175: invalid configuration argument.
